In [1]:
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats
from scipy.stats import t,norm
import numpy as np
import pandas as pd
import  statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import proportion
from scipy.stats import chisquare,chi2_contingency
import scipy
from scipy import stats
import pylab
from pylab import plot,show,legend,title,xlabel,ylabel
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_excel(r'E:\dbda khargar\Stats and R\diabetes.xlsx')

In [26]:
df.head()

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,DietType
0,148,72,35,0,33.6,0.63,50,1,1
1,85,66,29,0,26.6,0.35,31,0,2
2,183,64,0,0,23.3,0.67,32,1,1
3,89,66,23,94,28.1,0.17,21,0,1
4,137,40,35,168,43.1,2.29,33,1,1


In [8]:
df.shape

(768, 9)

In [6]:
df = df[['Glucose','DietType','Outcome']]

In [10]:
df.head()

,Glucose,DietType,Outcome
0,148,1,1
1,85,2,0
2,183,1,1
3,89,1,0
4,137,1,1


In [7]:
diet_dummy = pd.get_dummies(df['DietType'],drop_first=True).astype(int)
df = df.drop(['DietType'],axis=1)
df = pd.concat([df,diet_dummy],axis=1)
x_train,x_test,y_train,y_test=train_test_split(df.drop('Outcome',axis=1),df.Outcome,test_size=0.2)

In [24]:
df.head()

,Glucose,Outcome,2,3
0,148,1,False,False
1,85,0,True,False
2,183,1,False,False
3,89,0,False,False
4,137,1,False,False


In [8]:
x_train = sm.add_constant(x_train, prepend = False)
x_test = sm.add_constant(x_test, prepend=False)
mod1 = sm.Logit(y_train,x_train).fit()
print(mod1.summary())

Optimization terminated successfully.
         Current function value: 0.322803
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  614
Model:                          Logit   Df Residuals:                      610
Method:                           MLE   Df Model:                            3
Date:                Sat, 08 Jun 2024   Pseudo R-squ.:                  0.5023
Time:                        10:31:35   Log-Likelihood:                -198.20
converged:                       True   LL-Null:                       -398.21
Covariance Type:            nonrobust   LLR p-value:                 2.197e-86
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Glucose        0.0400      0.005      8.360      0.000       0.031       0.049
2             -4.0234      0.

In [9]:
mypred = mod1.predict(x_test)

In [10]:
y_pred= []
for ctr in mypred:
    if ctr>=0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [11]:
from sklearn.metrics import confusion_matrix
#this helps plot y_test against y_prod

In [13]:
print(confusion_matrix(y_test,y_pred))

[[97  5]
 [16 36]]


In [22]:
#accuracy of model 
100*((97+36)/((97+5)+(16+36)))

86.36363636363636

## Precison, Recall and f1 Score

In [27]:
#Accuracy is not a right test for a model instead to test the values we have precision
# when someone is falsifying the info
# We have precision for positive and precision for negative
# Precision for positive = TP / (TP + FP) , Precision for negative = TN / (TN + FN)
prec_p = 36/(36+5)
prec_n = 97/(16+97)
#Recall for positive = TP / (TP + FN), Recall for negative = TN / (TN + FP)
rec_p = 36/(36+5)
rec_n = 97/(97+5)
print(prec_p,prec_n,rec_p,rec_n)


0.8780487804878049 0.8584070796460177 0.8780487804878049 0.9509803921568627


In [26]:
# for calculation of precions and recalls 
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       102
           1       0.88      0.69      0.77        52

    accuracy                           0.86       154
   macro avg       0.87      0.82      0.84       154
weighted avg       0.87      0.86      0.86       154



In [28]:
#F1 score = 2*precision*recall/(precision+recall)


# Nominal Logistic Regression


## response in nominal scale so we use nominal logistic regression

In [2]:
df = pd.read_excel('CDAC_DataBook.xlsx',sheet_name='nominal')

In [3]:
df=df[['ses','math','prog']]
df.head()

,ses,math,prog
0,1,41,1
1,2,41,2
2,3,44,3
3,1,42,1
4,2,40,2


In [4]:
ses_dummy = pd.get_dummies(df['ses'],drop_first=True).astype(int)
df = df.drop(['ses'],axis=1)
df = pd.concat([df,ses_dummy],axis=1)
x_train,x_test,y_train,y_test=train_test_split(df.drop('prog',axis=1),df.prog,test_size=0.2)
x_train = sm.add_constant(x_train, prepend = False)
x_test = sm.add_constant(x_test, prepend = False)

from statsmodels.discrete import discrete_model
from statsmodels.discrete.discrete_model import MNLogit

mod2 = sm.MNLogit(y_train,x_train).fit()
print(mod2.summary())

Optimization terminated successfully.
         Current function value: 0.682969
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:                   prog   No. Observations:                  160
Model:                        MNLogit   Df Residuals:                      152
Method:                           MLE   Df Model:                            6
Date:                Mon, 10 Jun 2024   Pseudo R-squ.:                  0.3477
Time:                        09:51:42   Log-Likelihood:                -109.28
converged:                       True   LL-Null:                       -167.52
Covariance Type:            nonrobust   LLR p-value:                 8.913e-23
    prog=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
math           0.0046      0.033      0.139      0.890      -0.061       0.070
2              2.8929      0.

In [118]:
x_train.head()

,math,2,3,const
80,54,0,0,1.0
156,61,0,1,1.0
97,53,1,0,1.0
94,41,1,0,1.0
14,40,0,0,1.0


In [120]:
mydata = pd.DataFrame([[40,0,1,1],[60,0,1,1],[80,0,1,1]])
mydata.columns = x_train.columns
mod2.predict(mydata)

,0,1,2
0,0.053422,0.421774,0.524804
1,0.009574,0.077878,0.912549
2,0.001070,0.008971,0.989958


In [121]:
mydata = pd.DataFrame([[40,0,0,1],[60,0,0,1],[80,0,0,1]])
mod2.predict(mydata)

,0,1,2
0,0.653109,0.335452,0.011439
1,0.588528,0.311455,0.100018
2,0.313061,0.170703,0.516236


## When marks are same but you vary from ses 0 (low) to 1 (medium) to 2 (high)

In [17]:
mydata = pd.DataFrame([[70,0,0,1],[70,1,0,1],[70,0,1,1]])
mydata.columns=x_train.columns
mod2.predict(mydata)

,0,1,2
0,0.521766,0.207149,0.271085
1,0.020102,0.144021,0.835878
2,0.003200,0.024753,0.972047


In [9]:
mydata

,math,2,3,const
0,70,0,0,1
1,70,0,1,1
2,70,0,0,1


In [16]:
np.log(0.972047/0.003200)-np.log(0.271085/0.521766)

6.371040134072748